#### **Objetivo**
- Ler todos os arquivos CSV brutos separados, processa-los e mesclá-los em um único conjunto de dados

#### **Informações Complementares**
 - Os dados foram coletados usando um aplicativo de aquisição de dados instalado em um smart watch Apple.
- O smart watch contém tanto um acelerômetro quanto um giroscópio, que foram usados para coletar dados sobre os movimentos de exercícios.
- Os sensores foram configurados com as configurações padrão: o acelerômetro a 12.500Hz e o giroscópio a 25.000Hz.
- Cinco participantes foram recrutados para realizar exercícios básicos com barra em séries de 5 (heavy) e 10 (medium) repetições (supino: bench, levantamento terra: dead, desenvolvimento de ombro (militar): ohp, remada alta: row e agachamento livre: squat).
- O conjunto de dados bruto continham data/hora e valores x, y e z dos sensors (acelerômetro e giroscópio.
---



In [33]:
import pandas as pd
from glob import glob
import re

In [34]:
# --------------------------------------------------------------
# List all data in data/raw/MetaMotion
# --------------------------------------------------------------

files = glob("../data/raw/*.csv")
len(files)

164

In [35]:
# --------------------------------------------------------------
# Extract features from filename
# --------------------------------------------------------------

# Extracting 3 values of file name:
# participant,
# label (exercise) and
# categoty (light or heavy)

f = files[0]
f.split("-")[0]
f.split("-")[1]
f.split("-")[2].split("_")[0]

data_path = "../data/raw\\"

# Extrating the participant
participant = f.split("-")[0].replace(data_path, "")
# Extracting the label
label = f.split("-")[1]

# Removing number in any string
def remove_numbers(string):
    return re.sub(r"\d+", "", string)


category = remove_numbers(f.split("-")[2].split("_")[0])  # Extracting the category

# Adding extra columns
df = pd.read_csv(f)
df["participant"] = f.split("-")[0].replace(data_path, "")
df["label"] = f.split("-")[1]
df["category"] = remove_numbers(f.split("-")[2].split("_")[0])

df.head(3)

,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),participant,label,category
0,1547219408431,2019-01-11T16:10:08.431,0.00,0.010,0.964,-0.087,A,bench,heavy
1,1547219408511,2019-01-11T16:10:08.511,0.08,0.000,0.961,-0.069,A,bench,heavy
2,1547219408591,2019-01-11T16:10:08.591,0.16,0.001,0.974,-0.087,A,bench,heavy


In [36]:
# --------------------------------------------------------------
# Read all files
# --------------------------------------------------------------

# Create df to store acc anf gyr
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

# Creating a set to increment alfter each file is read
acc_set = 1
gyr_set = 1

# Lopping through all files
for f in files:
    # Reading the file
    df = pd.read_csv(f)
    # Extracting the participant
    participant = f.split("-")[0].replace(data_path, "")
    # Extracting the label
    label = f.split("-")[1]
    # Removing number in any string
    category = remove_numbers(f.split("-")[2].split("_")[0])  # Extracting the category
    df["participant"] = participant
    df["label"] = label
    df["category"] = category
    # If exist the word 'accelerometer' in the file name
    if "Accelerometer" in f:
        # Adding to acc_df
        df["set"] = acc_set
        acc_df = pd.concat([acc_df, df])
        acc_set += 1
    if "Gyroscope" in f:
        # Adding to gyr_df
        df["set"] = gyr_set
        gyr_df = pd.concat([gyr_df, df])
        gyr_set += 1

display(df.head(3))
display(df.tail(3))

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category,set
0,1547666043834,2019-01-16 20:14:03.834,0.00,-10.549,1.646,-1.280,E,squat,heavy,82
1,1547666043874,2019-01-16 20:14:03.874,0.04,-8.963,1.524,-0.732,E,squat,heavy,82
2,1547666043914,2019-01-16 20:14:03.914,0.08,-2.378,-1.280,0.305,E,squat,heavy,82


,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category,set
489,1547666063394,2019-01-16 20:14:23.394,19.56,5.854,-2.561,1.463,E,squat,heavy,82
490,1547666063434,2019-01-16 20:14:23.434,19.60,4.268,0.549,-0.305,E,squat,heavy,82
491,1547666063474,2019-01-16 20:14:23.474,19.64,-1.098,1.768,-0.122,E,squat,heavy,82


In [37]:
# --------------------------------------------------------------
# Working with datetimes
# --------------------------------------------------------------
display(acc_df.info())

# Converting UNIX time to datetime
pd.to_datetime(df["epoch (ms)"], unit="ms")

<class 'pandas.core.frame.DataFrame'>
Index: 20334 entries, 0 to 241
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   epoch (ms)    20334 non-null  int64  
 1   time (01:00)  20334 non-null  object 
 2   elapsed (s)   20334 non-null  float64
 3   x-axis (g)    20334 non-null  float64
 4   y-axis (g)    20334 non-null  float64
 5   z-axis (g)    20334 non-null  float64
 6   participant   20334 non-null  object 
 7   label         20334 non-null  object 
 8   category      20334 non-null  object 
 9   set           20334 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 1.7+ MB


None

0     2019-01-16 19:14:03.834
1     2019-01-16 19:14:03.874
2     2019-01-16 19:14:03.914
3     2019-01-16 19:14:03.954
4     2019-01-16 19:14:03.994
                ...          
487   2019-01-16 19:14:23.314
488   2019-01-16 19:14:23.354
489   2019-01-16 19:14:23.394
490   2019-01-16 19:14:23.434
491   2019-01-16 19:14:23.474
Name: epoch (ms), Length: 492, dtype: datetime64[ns]

In [38]:
# Conveting object to datetime
pd.to_datetime(df["time (01:00)"]).dt.weekday

0      2
1      2
2      2
3      2
4      2
      ..
487    2
488    2
489    2
490    2
491    2
Name: time (01:00), Length: 492, dtype: int32

In [39]:
# Set time to index
acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit="ms")
gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit="ms")

# Removing columns
del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]

# A single exemple
acc_df.query('participant == "E"').query('label == "row" ').query(
    'category == "medium" '
)

,x-axis (g),y-axis (g),z-axis (g),participant,label,category,set
epoch (ms),,,,,,,
2019-01-19 17:30:49.220,-0.051,-1.026,-0.088,E,row,medium,78
2019-01-19 17:30:49.300,-0.054,-1.038,-0.093,E,row,medium,78
2019-01-19 17:30:49.380,-0.052,-1.034,-0.095,E,row,medium,78
2019-01-19 17:30:49.460,-0.063,-1.011,-0.095,E,row,medium,78
2019-01-19 17:30:49.540,-0.052,-1.017,-0.090,E,row,medium,78
...,...,...,...,...,...,...,...
2019-01-19 17:35:13.381,-0.060,-1.021,-0.058,E,row,medium,80
2019-01-19 17:35:13.461,-0.035,-1.037,-0.026,E,row,medium,80
2019-01-19 17:35:13.541,-0.045,-1.029,-0.033,E,row,medium,80


In [41]:
# --------------------------------------------------------------
# Turn into a single function (read_data_from_files)
# --------------------------------------------------------------

files = glob("../data/raw/*.csv")
data_path = "../data/raw\\"

# Removing number in any string
def remove_numbers(string):
    return re.sub(r"\d+", "", string)


def read_data_from_files(files: list, data_path: str) -> tuple:
    """
    Read data from a list of files and extract & clean accelerometer and gyroscope data.

    Args:
        files (list): List of file paths to read data from.
        data_path (str): Path to the data directory.

    Returns:
        tuple: A tuple containing two DataFrames - `acc_df` (accelerometer data) and `gyr_df` (gyroscope data).
    """
    # Create empty DataFrames to store accelerometer and gyroscope data
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame()

    # Set initial values for set counters
    acc_set = 1
    gyr_set = 1

    # Loop through all files
    for f in files:
        # Reading the file
        df = pd.read_csv(f)

        # Extracting participant, label, and category from the file name
        participant = f.split("-")[0].replace(data_path, "")
        label = f.split("-")[1]
        category = remove_numbers(f.split("-")[2].split("_")[0])

        # Add participant, label, and category columns to the DataFrame
        df["participant"] = participant
        df["label"] = label
        df["category"] = category

        # Check if the file contains accelerometer data
        if "Accelerometer" in f:
            # Add set number to the DataFrame
            df["set"] = acc_set
            # Concatenate the DataFrame to the acc_df
            acc_df = pd.concat([acc_df, df])
            # Increment the acc_set counter
            acc_set += 1

        # Check if the file contains gyroscope data
        if "Gyroscope" in f:
            # Add set number to the DataFrame
            df["set"] = gyr_set
            # Concatenate the DataFrame to the gyr_df
            gyr_df = pd.concat([gyr_df, df])
            # Increment the gyr_set counter
            gyr_set += 1

    # Set time as the DataFrame index
    acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit="ms")
    gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit="ms")

    # Remove unnecessary columns from the DataFrames
    columns_to_remove = ["epoch (ms)", "time (01:00)", "elapsed (s)"]
    acc_df.drop(columns_to_remove, axis=1, inplace=True)
    gyr_df.drop(columns_to_remove, axis=1, inplace=True)

    return acc_df, gyr_df


acc_df, gyr_df = read_data_from_files(files, data_path)

display(acc_df)
display(gyr_df)

,x-axis (g),y-axis (g),z-axis (g),participant,label,category,set
epoch (ms),,,,,,,
2019-01-11 15:10:08.431,0.010,0.964,-0.087,A,bench,heavy,1
2019-01-11 15:10:08.511,0.000,0.961,-0.069,A,bench,heavy,1
2019-01-11 15:10:08.591,0.001,0.974,-0.087,A,bench,heavy,1
2019-01-11 15:10:08.671,-0.012,0.971,-0.084,A,bench,heavy,1
2019-01-11 15:10:08.751,-0.013,0.954,-0.094,A,bench,heavy,1
...,...,...,...,...,...,...,...
2019-01-16 19:14:23.089,0.012,0.596,0.815,E,squat,heavy,82
2019-01-16 19:14:23.169,0.009,0.528,0.821,E,squat,heavy,82
2019-01-16 19:14:23.249,0.015,0.554,0.746,E,squat,heavy,82


,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category,set
epoch (ms),,,,,,,
2019-01-11 15:10:08.351,0.122,-5.488,-3.841,A,bench,heavy,1
2019-01-11 15:10:08.391,2.195,-9.695,-0.610,A,bench,heavy,1
2019-01-11 15:10:08.431,2.622,-8.110,-4.024,A,bench,heavy,1
2019-01-11 15:10:08.471,1.951,-4.695,-4.634,A,bench,heavy,1
2019-01-11 15:10:08.511,1.524,-2.561,-2.500,A,bench,heavy,1
...,...,...,...,...,...,...,...
2019-01-16 19:14:23.314,-1.707,-0.671,1.585,E,squat,heavy,82
2019-01-16 19:14:23.354,0.915,0.305,2.988,E,squat,heavy,82
2019-01-16 19:14:23.394,5.854,-2.561,1.463,E,squat,heavy,82


In [54]:
# --------------------------------------------------------------
# Merging datasets in a single dataframe
# --------------------------------------------------------------

# Selecting just first 3 columns from acc_df
data_merged = pd.concat([acc_df.iloc[:, :3], gyr_df], axis=1)

# Rename columns
data_merged.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
]

# Dropping the nan values
# There are two sensors measuring at diferent frequencies
# The change that the sensors measurement exact at same time is small
data_merged
data_merged.dropna(inplace=False).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1119 entries, 2019-01-11 15:10:08.431000 to 2019-01-15 19:04:25.881000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        1119 non-null   float64
 1   acc_y        1119 non-null   float64
 2   acc_z        1119 non-null   float64
 3   gyr_x        1119 non-null   float64
 4   gyr_y        1119 non-null   float64
 5   gyr_z        1119 non-null   float64
 6   participant  1119 non-null   object 
 7   label        1119 non-null   object 
 8   category     1119 non-null   object 
 9   set          1119 non-null   float64
dtypes: float64(7), object(3)
memory usage: 96.2+ KB


In [55]:
# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz

aggregation = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last",
    "label": "last",
    "category": "last",
    "set": "last",
}

# Resample the DataFrame
data_merged[:1000].resample(rule="200ms").agg(aggregation, errors="ignore")

# Generates a list called df_by_days, where each element represents a DataFrame
# for a specific day. The groupby() operation allows you to group the original DataFrame
# data_merged by day, and then the list comprehension extracts and collects the
# DataFrames into the df_by_days list
df_by_days = [df_by_day for day, df_by_day in data_merged.groupby(pd.Grouper(freq="D"))]

# Takes the list of DataFrames df_by_days, performs resampling and aggregation on each
# DataFrame, drops missing values, and then concatenates the resampled and aggregated
# DataFrames into a single DataFrame called data_resampled.
data_resampled = pd.concat(
    [
        df.resample(rule="200ms").agg(aggregation, errors="ignore").dropna()
        for df in df_by_days
    ]
)

data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8180 entries, 2019-01-11 15:08:05.200000 to 2019-01-19 17:35:13.600000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        8180 non-null   float64
 1   acc_y        8180 non-null   float64
 2   acc_z        8180 non-null   float64
 3   gyr_x        8180 non-null   float64
 4   gyr_y        8180 non-null   float64
 5   gyr_z        8180 non-null   float64
 6   participant  8180 non-null   object 
 7   label        8180 non-null   object 
 8   category     8180 non-null   object 
 9   set          8180 non-null   float64
dtypes: float64(7), object(3)
memory usage: 703.0+ KB


In [57]:
# --------------------------------------------------------------
# Turn into a single function (resample_and_aggregate_data)
# --------------------------------------------------------------

def resample_and_aggregate_data(
    acc: pd.DataFrame,
    gyr: pd.DataFrame,
    columns_name: list,
    rule: str,
    aggregation: dict,
) -> pd.DataFrame:
    """
    Resample and aggregate accelerometer and gyroscope data based on a specified rule and 
    aggregation dictionary.

    Args:
        acc (pd.DataFrame): The accelerometer data.
        gyr (pd.DataFrame): The gyroscope data.
        columns_name (list): The column names for the merged DataFrame.
        rule (str): The resampling rule.
        aggregation (dict): The aggregation dictionary specifying the columns and 
        aggregation methods.

    Returns:
        pd.DataFrame: The resampled and aggregated DataFrame.
    """
    # Selecting just the first 3 columns from acc_df and combining with gyr_df
    data_merged = pd.concat([acc.iloc[:, :3], gyr], axis=1)

    # Rename columns
    data_merged.columns = columns_name

    # Group by day
    df_by_days = [
        df_by_day for day, df_by_day in data_merged.groupby(pd.Grouper(freq="D"))
    ]

    # Resample and concate each day's DataFrame
    data_resampled = pd.concat(
        [
            df.resample(rule=rule).agg(aggregation, errors="ignore").dropna()
            for df in df_by_days
        ]
    )

    data_resampled["set"] = data_resampled["set"].astype("int")

    return data_resampled



In [58]:
columns_name = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
]

aggregation = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last",
    "label": "last",
    "category": "last",
    "set": "last",
}

rule = "200ms"

data_resampled = resample_and_aggregate_data(
    acc_df, gyr_df, columns_name, rule, aggregation
)

data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8180 entries, 2019-01-11 15:08:05.200000 to 2019-01-19 17:35:13.600000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        8180 non-null   float64
 1   acc_y        8180 non-null   float64
 2   acc_z        8180 non-null   float64
 3   gyr_x        8180 non-null   float64
 4   gyr_y        8180 non-null   float64
 5   gyr_z        8180 non-null   float64
 6   participant  8180 non-null   object 
 7   label        8180 non-null   object 
 8   category     8180 non-null   object 
 9   set          8180 non-null   int32  
dtypes: float64(6), int32(1), object(3)
memory usage: 671.0+ KB


**Códigos orignais:** `../src/data/make_dataset.py`

**Arquivo salvo em:** `../data/interim/01_data_resampled.pkl"`

**Próxima etapa:** `02_visualize.ipynb`